# Using AutoGen's Selector Group Chat for Multi-Agent Collaboration

This lesson demonstrates how to use AutoGen's SelectorGroupChat to create a dynamic multi-agent system where agents collaborate to solve complex tasks. We'll create a sports statistics analysis system as an example.

In [1]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

In [ ]:
from typing import Sequence
from autogen import AssistantAgent, SelectorGroupChat
from autogen.agentchat.contrib.text_analyzer_agent import TextAnalyzerAgent
from autogen.agentchat.conditions import MaxMessageTermination, TextMentionTermination

# Mock tools for demonstration
def search_sports_stats(query: str) -> str:
    stats_db = {
        "lebron_2020": "LeBron James 2020-21 stats: 25.0 PPG, 7.7 RPG, 7.8 APG",
        "curry_2021": "Stephen Curry 2021-22 stats: 25.5 PPG, 5.2 RPG, 6.3 APG",
        "durant_2022": "Kevin Durant 2022-23 stats: 29.1 PPG, 6.7 RPG, 5.0 APG"
    }
    return stats_db.get(query.lower().replace(" ", "_"), "No stats found.")

def calculate_efficiency(points: float, rebounds: float, assists: float) -> float:
    return (points + rebounds * 1.2 + assists * 1.5) / 3

# Create specialized agents
planning_agent = AssistantAgent(
    name="PlanningAgent",
    description="Strategic coordinator that breaks down tasks and manages workflow",
    system_message="""You are a planning specialist. Your role is to:
    1. Break down complex tasks into subtasks
    2. Assign tasks to appropriate team members
    3. Monitor progress and ensure task completion
    4. Summarize findings
    
    End your final summary with 'TASK_COMPLETE'."""
)

stats_agent = AssistantAgent(
    name="StatsAgent",
    description="Sports statistics expert with access to historical data",
    system_message="You are a sports statistics expert. Use the search_sports_stats tool to find player statistics.",
    tools=[search_sports_stats]
)

analysis_agent = AssistantAgent(
    name="AnalysisAgent", 
    description="Analytics specialist that processes statistical data",
    system_message="You are an analytics expert. Use the calculate_efficiency tool to analyze player performance.",
    tools=[calculate_efficiency]
)

# Create termination conditions
termination = TextMentionTermination("TASK_COMPLETE") | MaxMessageTermination(max_messages=15)

# Custom selector function to ensure Planning Agent reviews after each action
def selector_func(messages: Sequence[str]) -> str | None:
    if messages[-1].source != planning_agent.name:
        return planning_agent.name
    return None

# Initialize the group chat
team = SelectorGroupChat(
    agents=[planning_agent, stats_agent, analysis_agent],
    selector_func=selector_func,
    termination_condition=termination,
    max_round=5
)

# Example task
async def analyze_player_performance(player_name: str, season: str):
    task = f"Analyze {player_name}'s performance in the {season} season. Calculate their efficiency rating."
    result = await team.run(task)
    return result

# Usage example

import asyncio

async def main():
    result = await analyze_player_performance("LeBron James", "2020-21")
    print(result)

asyncio.run(main())

This example demonstrates several key concepts:

1. **Agent Specialization**: We create three specialized agents:
   - Planning Agent: Coordinates the workflow
   - Stats Agent: Handles data retrieval
   - Analysis Agent: Processes statistics

2. **Custom Tools**: We implement mock tools for demonstration:
   - `search_sports_stats`: Simulates a sports statistics database
   - `calculate_efficiency`: Computes a player efficiency rating

3. **Selector Function**: A custom selector ensures the Planning Agent reviews after each action:

In [ ]:
def selector_func(messages: Sequence[str]) -> str | None:
    if messages[-1].source != planning_agent.name:
        return planning_agent.name
    return None

4. **Termination Conditions**: We use two conditions:
   - Text mention of "TASK_COMPLETE"
   - Maximum message limit of 15

5. **Group Chat Configuration**: The SelectorGroupChat is initialized with:
   - List of agents
   - Custom selector function
   - Termination conditions
   - Maximum rounds

To use this system:

In [ ]:
# Example task execution
async def main():
    result = await analyze_player_performance("LeBron James", "2020-21")
    print(result)

asyncio.run(main())

The agents will collaborate to:
1. Retrieve player statistics
2. Calculate efficiency ratings
3. Provide analysis and insights
4. Summarize findings

This implementation demonstrates how AutoGen's SelectorGroupChat can create sophisticated multi-agent systems with dynamic task allocation and structured workflows.